<a href="https://colab.research.google.com/github/felipefernandes/lisa/blob/main/LiSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
!pip install -q -U google-generativeai
!pip install -q -U pathlib

In [99]:
import pathlib
import hashlib
import google.generativeai as genai
from google.colab import userdata, drive
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import Markdown
import random

In [100]:
# Configuração do GenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINIKEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [101]:
# Listando modelos disponíveis
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [102]:
# Documentos

estruturas = [
    {"nome": "Impromptu Networking", "duracao": "5-20 min", "proposito": "Compartilhe rapidamente os desafios e as expectativas, crie novas conexões", "label": "Disseminar"},
    {"nome": "9 Whys", "duracao": "5-20 min", "proposito": "Torne claro o propósito do trabalho juntos", "label": "Analisar"},
    {"nome": "What, So What, Now What", "duracao": "15-45 min", "proposito": "Analisar juntos o progresso até então e decidir quais ajustes são necessários", "label": "Analisar"},
    {"nome": "TRIZ", "duracao": "30-45 min", "proposito": "Pare atividades e comportamentos contraproducentes para abrir espaço para a inovação", "label": "Revelar"},
    {"nome": "Appreciative Interviews", "duracao": "30-60 min", "proposito": "Descobrindo e aproveitando as causas-raiz do sucesso", "label": "Revelar, Disseminar"},
    {"nome": "1-2-4-All", "duracao": "10-12 min", "proposito": "Engage todos simultaneamente na geração de perguntas, ideias e sugestões.", "label": "Revelar"},
    {"nome": "User Experience Fishbowl", "duracao": "25-70 min", "proposito": "Compartilhe com uma comunidade maior a capacidade obtida a partir da experiência", "label": "Disseminar"},
    {"nome": "15% Solutions", "duracao": "15-20 min", "proposito": "Descubra e concentre-se no que cada pessoa tem a liberdade e os recursos para fazer agora", "label": "Revelar"},
    {"nome": "25-to-10 Crowdsourcing", "duracao": "20-30 min", "proposito": "Gerar e analisar rapidamente as ideias factíveis mais poderosas de um grupo", "label": "Revelar"},
    { "nome": "Troika Consulting", "duracao": "15-30 min", "proposito": "Obtenha ajuda prática e criativa dos colegas imediatamente", "label": "Ajuda, Revelar"},
    { "nome": "Conversation Café", "duracao": "35-60 min", "proposito": "Engage todos na compreensão de grandes desafios", "label": "Disseminar"},
    { "nome": "Min Specs", "duracao": "20-50 min", "proposito": "Especifique apenas os absolutos “Devemos” e “Não Devemos” para alcançar um propósito", "label": "Estratégia"},
    { "nome": "Wise Crowds", "duracao": "10-60 min. per person", "proposito": "Acesse a sabedoria do grupo inteiro em ciclos rápidos", "label": "Ajuda, Revelar"},
    { "nome": "Wicked Questions", "duracao": "20 min", "proposito": "Articule os desafios paradoxais que um grupo deve enfrentar para ter sucesso", "label": "Revelar"},
    { "nome": "Drawing Together", "duracao": "30-40 min", "proposito": "Revele ideias e caminhos para avançar através da expressão não-verbal", "label": "Revelar"},
    { "nome": "Improv Prototyping", "duracao": "15-20 min. per round", "proposito": "Desenvolva soluções eficazes para desafios crônicos, enquanto se diverte seriamente", "label": "Revelar"},
    { "nome": "Agreement-Certainty Matrix", "duracao": "30-45 min", "proposito": "Classifique os desafios nos domínios simples, complicado, complexo e caótico", "label": "Análise"},
    { "nome": "Shift & Share", "duracao": "35-90 min", "proposito": "Dissemine boas ideias e faça conexões informais com inovadores", "label": "Disseminar"},
    { "nome": "Heard, Seen, Respected", "duracao": "25 min", "proposito": "Pratique a escuta profunda e empatia com colegas", "label": "Ajuda"},
    { "nome": "Social Network Webbing", "duracao": "45-60 min", "proposito": "Mapeie conexões informais e decida como fortalecer a rede para alcançar um propósito", "label": "Ajuda, Disseminar"},
    { "nome": "Design Storyboards", "duracao": "25-70 min", "proposito": "Defina os elementos passo-a-passo para conduzir reuniões a finais produtivos", "label": "Planejar"},
    { "nome": "Open Space", "duracao": "90 min. to 3 days", "proposito": "Libere a ação e liderança inerentes a grupos de qualquer tamanho", "label": "Revelar"},
    { "nome": "Discovery & Action Dialogue", "duracao": "25-70 min", "proposito": "Descubra, invente e desencadeie soluções locais para problemas crônicos", "label": "Revelar"},
    { "nome": "Integrated Autonomy", "duracao": "60-80 min", "proposito": "Mude de soluções 'ou-um-ou-outro' para robustas soluções 'ambos-e'", "label": "Estratégia"},
    { "nome": "Generative Relationships", "duracao": "25 min", "proposito": "Revele padrões de relacionamento que criam surpreendente valor ou disfunções", "label": "Análise"},
    { "nome": "Critical Uncertainties", "duracao": "60-100 min", "proposito": "Desenvolva estratégias para operar em uma variedade de futuros plausíveis porém imprevisíveis", "label": "Estratégia"},
    { "nome": "Purpose-to-Practice", "duracao": "25-120 min", "proposito": "Desenhe os cinco elementos essenciais para uma iniciativa resiliente e duradoura", "label": "Planejar"},
    { "nome": "Ecocycle Planning", "duracao": "60-95 min", "proposito": "Analise o portfólio completo de atividades e relacionamentos para identificar obstáculos e oportunidades para o progresso", "label": "Análise, Estratégia"},
    { "nome": "Panarchy", "duracao": "1-2 hr", "proposito": "Entenda como sistemas interconectados interagem, evoluem, disseminam inovação e transformam", "label": "Estratégia"},
    { "nome": "What I Need From You", "duracao": "45-70 min", "proposito": "Faça emergir as necessidades essenciais entre áreas e aceite ou rejeite pedidos de suporte", "label": "Ajuda"},
    { "nome": "Celebrity Interview", "duracao": "25-60 min", "proposito": "Reconectar a experiência de líderes e especialistas com pessoas mais próximas do desafio em mãos", "label": "Disseminar"},
    { "nome": "Helping Heuristics", "duracao": "15 min", "proposito": "Pratique métodos progressivos para ajudar os outros, receber ajuda e pedir ajuda", "label": "Ajuda"},
    { "nome": "Simple Ethnography", "duracao": "1-6 hr", "proposito": "Observe e registre os comportamentos reais dos usuários em campo", "label": "Análise, Revelar"}
]

solucoes = [
    {"solucao": "Revelar, gerar aprimorar idéias ou soluções"},
    {"solucao": "Compartilhar ou disseminar idéias, conhecimentos ou experiências"},
    {"solucao": "Analisar, diagnosticar ou processar"},
    {"solucao": "Pedir ou oferecer ajuda"},
    {"solucao": "Elaborar estratégias"},
    {"solucao": "Planejar"}
]

In [103]:
df_estruturas = pd.DataFrame(estruturas)
df_estruturas.columns = ['nome', 'duracao', 'proposito', 'label']

df_solucoes = pd.DataFrame(solucoes)
df_solucoes.columns = ['solucao']

In [104]:
# Inicializando o modelo
model = "models/embedding-001"

In [105]:
# Função para adicionar colunas de Embeds no DF Estruturas, conteudo principal titulo e descricao
def addEmbeddings(nome, proposito):
  return genai.embed_content(model=model, content=proposito, title=nome, task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Adicionando Embeddings no DataFrame
df_estruturas["Embeddings"] = df_estruturas.apply(lambda row: addEmbeddings(row["nome"], row["proposito"]), axis=1)

In [106]:
# Função para adicionar uma coluna de Embeddings para Label
def addLabelEmbeddings(label):
  return genai.embed_content(model=model, content=label, task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Adicionando nova coluna de Embeddings para Label
df_estruturas["LabelEmbeddings"] = df_estruturas.apply(lambda row: addLabelEmbeddings(row["label"]), axis=1)

In [107]:
# Função para adicionar colunas embeddings no DF de Solucoes
def addSolucaoEmbeddings(solucao):
  return genai.embed_content(model=model, content=solucao, task_type="RETRIEVAL_DOCUMENT")["embedding"]

df_solucoes["Embeddings"] = df_solucoes.apply(lambda row: addSolucaoEmbeddings(row["solucao"]), axis=1)

In [108]:
# Função para verificar consulta tem relação com alguma dos itens do DF solução
def buscar_solucao_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(base['Embeddings']), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return base.iloc[indice]['solucao']


# Limite de Relevancia

* O `limite_relevancia` atua como um filtro, determinando quais resultados são considerados "relevantes" com base no produto escalar entre o embedding da consulta e os embeddings da sua base de dados (`df_estruturas`).
* Um produto escalar próximo de 1 indica alta similaridade entre os vetores, enquanto um valor próximo de 0 indica baixa similaridade.

## Ajustando o Limite de Relevancia:
* **Aumentar o limite (mais perto de 1)**:
  * Somente resultados **muito similares** à consulta serão considerados relevantes.
  * Útil se você deseja resultados **precisos** e tem certeza de que a consulta corresponde bem aos dados na sua base.
  * Risco: Pode resultar em **nenhum resultado** se a consulta for muito específica ou não houver correspondência exata na base.
* **Diminuir o limite (mais perto de 0)**:
  * Resultados **menos similares** à consulta também serão considerados relevantes.
  * Útil para **explorar** a base de dados ou quando a consulta é mais vaga.
  * Risco: Pode retornar **resultados irrelevantes** ou que não correspondem exatamente à intenção da consulta.

In [122]:
# Função para gerar e buscar e consultar
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df_estruturas['LabelEmbeddings']), embedding_da_consulta)

  limite_relevancia = 0.6 # Definindo o limite da relevancia
  indices_relevantes = np.where(produtos_escalares >= limite_relevancia)[0]

  resultados_relevantes = []
  for indice in indices_relevantes:
    resultados_relevantes.append(df_estruturas.iloc[indice]['nome'])

  top3_resultados_relevantes = []
  top3 = 0
  if resultados_relevantes:
    while top3 < 3 and top3 < len(resultados_relevantes):  # Verifica se há elementos suficientes
      # Exibe a primeira posicao de resultados_relevante
      top3_resultados_relevantes.append(resultados_relevantes[top3])
      top3 += 1
    return top3_resultados_relevantes # Exibe uma lista de resultados relevantes
  else:
    indice = np.argmax(produtos_escalares)
    return df_estruturas.iloc[indice]['nome'] # garante que ao menos 1 resultado seja exibido

In [110]:
# Busca e retorna solucoes e ELs recomendadas de acordo com um Prompt
def busca (prompt):
  solucao_relacionada = buscar_solucao_consulta(prompt, df_solucoes, model)
  estruturas_recomendadas = gerar_e_buscar_consulta(prompt, df_estruturas, model)
  estruturas_recomendadas = ', '.join(estruturas_recomendadas)
  resposta = "Para o prompt: " + prompt + "\n" + "Solução identificada: " + solucao_relacionada + "\n" + "Estruturas recomendadas: " + estruturas_recomendadas + "\n"
  return resposta

# Geração de um Roteiro 📓
## Partes de um roteiro
* Abertura
* Geração de idéias e Reflexão
* Fechamento (ações)

### Abertura: Quais as ELs mais indicadas? ⚡

*Premissas:*
* As com duração < 15 min
* Revelar, Disseminar

ELs com duração de até 15 min:
* 1-2-4-All
* Impromptu Networking
* Wise Crowds

### Geração de idéias e Reflexão: Quais as ELs mais indicadas? ⚡

*Premissas:*
* Refletir, Analisar, Processamento, Revelar

ELs com duração de até 15 min:
* Shift and Share


### Fechamento: Quais as ELs mais indicadas? ⚡

*Premissas:*
* Revelar, disseminar, elaborar estratégias, planejar

ELs recomendadas:
* 15% Solutions
* Min Specs
* Conversation Café






---
* 1-2-4-All
* 15% Solutions
* Helping Heuristics
* Impromptu Networking
* Improv Prototyping
* Nine Whys
* Wise Crowds

In [111]:
# Geração de Roteiro
#
# Definição de dicionário de dados para seções do Roteiro

# Definição das atividades para cada seção
abertura_atividades = {
    "Fala 5 min": {
        "descricao": "Cada pessoa compartilha algo que tem feito para se distrair.",
        "formacao": "Individual",
        "duracao": 5
    },
    "Check-in": {
        "descricao": "Cada pessoa compartilha em uma palavra como está se sentindo.",
        "formacao": "Individual",
        "duracao": 5
    },
    "Desenho colaborativo": {
        "descricao": "Em duplas, as pessoas se revezam completando um desenho.",
        "formacao": "Duplas",
        "duracao": 10
    }
}

reflexao_atividades = {
    "1-2-4-All": {
        "descricao": "Brainstorming em etapas: individual, duplas, quartetos, e por fim, todos juntos.",
        "formacao": "Variada",
        "duracao": 20
    },
    "Mad Sad Glad": {
        "descricao": "As pessoas escrevem o que as deixou felizes, tristes e irritadas durante a sprint.",
        "formacao": "Individual",
        "duracao": 10
    },
    "Linha do Tempo": {
        "descricao": "Criar uma linha do tempo da sprint e marcar os principais eventos.",
        "formacao": "Grupo",
        "duracao": 15
    }
}

decisao_atividades = {
    "Dot Voting": {
        "descricao": "Votar nos tópicos mais importantes para serem discutidos.",
        "formacao": "Individual",
        "duracao": 5
    },
    "5 Porquês": {
        "descricao": "Investigar a causa raiz de um problema.",
        "formacao": "Grupo",
        "duracao": 15
    },
    "Plano de Ação": {
        "descricao": "Definir ações concretas para resolver os problemas priorizados.",
        "formacao": "Grupo",
        "duracao": 20
    }
}

fechamento_atividades = {
    "Check-out": {
        "descricao": "Cada pessoa compartilha em uma palavra como se sente após a retrospectiva.",
        "formacao": "Individual",
        "duracao": 5
    },
    "Um Agradecimento": {
        "descricao": "Cada pessoa agradece a outra por algo específico que contribuiu para a sprint.",
        "formacao": "Individual",
        "duracao": 10
    },
    "Próximos Passos": {
        "descricao": "Revisar o plano de ação e garantir que todos estejam cientes dos próximos passos.",
        "formacao": "Grupo",
        "duracao": 5
    }
}

In [112]:
# Função para encontrar ELs com base nos LabelEmbeddings
def encontrar_estruturas_por_embeddings(embeddings, df, top_n=3):
    # Calcula a similaridade entre os embeddings do prompt e os embeddings dos labels das ELs
    similaridades = model.docvecs.similarity_query(embeddings, documents=df["label"].tolist())

    # Ordena as ELs por similaridade e seleciona as top_n
    top_indices = sorted(range(len(similaridades)), key=lambda i: similaridades[i], reverse=True)[:top_n]
    return df.iloc[top_indices]

In [113]:
# Função para gerar um roteiro
def gerar_roteiro(prompt):
    roteiro = {}

    # Usar a função busca para obter sugestões de ELs
    resultado_busca = busca(prompt)
    estruturas_recomendadas_nomes = resultado_busca.split("Estruturas recomendadas: ")[1].strip().split(", ")

    # Encontrar as ELs no DataFrame usando os nomes recomendados
    estruturas_recomendadas = df_estruturas[df_estruturas["nome"].isin(estruturas_recomendadas_nomes)]

    # ----> Correção: Converter o DataFrame para dicionário
    estruturas_dict = df_estruturas.set_index('nome').to_dict(orient='index')

    # Selecionar as ELs adequadas para cada seção
    reflexao_estrutura = None
    decisao_estrutura = None

    for estrutura_nome in estruturas_recomendadas_nomes:
        if estrutura_nome in estruturas_dict:
            estrutura_data = estruturas_dict[estrutura_nome]
            if estrutura_data["label"] in ["Revelar", "Disseminar", "Compartilhar"]:
                reflexao_estrutura = estrutura_data
            elif estrutura_data["label"] in ["Analisar", "Diagnosticar", "Processar", "Elaborar", "Planejar"]:
                decisao_estrutura = estrutura_data

    # Adicionar as atividades ao roteiro
    roteiro["Abertura"] = random.choice(list(abertura_atividades.items()))

    if reflexao_estrutura:
        roteiro["Reflexão e Colhendo Dados"] = (reflexao_estrutura["nome"], {
            "descricao": reflexao_estrutura["proposito"],
            "formacao": "Variável", # A formação depende da EL específica
            "duracao": reflexao_estrutura["duracao"]
        })
    else:
        roteiro["Reflexão e Colhendo Dados"] = random.choice(list(reflexao_atividades.items()))

    if decisao_estrutura:
        roteiro["Decidindo o que fazer"] = (decisao_estrutura["nome"], {
            "descricao": decisao_estrutura["proposito"],
            "formacao": "Variável", # A formação depende da EL específica
            "duracao": decisao_estrutura["duracao"]
        })
    else:
        roteiro["Decidindo o que fazer"] = random.choice(list(decisao_atividades.items()))

    roteiro["Fechamento"] = random.choice(list(fechamento_atividades.items()))

    return roteiro, resultado_busca

In [114]:
def print_roteiro(roteiro):
  print("# Retrospectiva (roteiro)\n")

  for secao, (atividade, detalhes) in roteiro.items():
      print(f"## {secao}")
      print(f"**Duração:** {detalhes['duracao']} min\n")
      print(f"**Formação:** {detalhes['formacao']}\n")
      print(f"**Atividade:** {atividade}\n")
      print(f"{detalhes['descricao']}")
      print()

In [124]:
# Preparando e instanciando o modo generativo para respostas
model_generative_config = {
    "temperature": 2,
    "candidate_count": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

model_generative_safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Seu nome é Lisa. Uma assistente virtual, com habilidades de um Coach profissional e experiente facilitadora de reuniões e atividades presenciais ou online com amplo conhecimento de aplicação de Estruturas Libertadoras. Sempre pergunte ao usuário se ele deseja mais alguma informação e o lembre que a palavra-chave para terminar a conversa é 'fim'"

model_generative = genai.GenerativeModel("gemini-1.5-pro-latest",
                                         generation_config=model_generative_config,
                                         system_instruction=system_instruction,
                                         safety_settings=model_generative_safety_settings)

In [116]:
chat = model_generative.start_chat(history=[
    {
      "role": "user",
      "parts": ["Se apresente."]
    },
])

In [117]:
def update_chat_history(chat, role, message):
  """Adiciona uma nova entrada ao histórico da conversa.

  Args:
    chat: Objeto de chat do modelo generativo.
    role: A função na conversa ("user" ou "assistant").
    message: A mensagem a ser adicionada.
  """
  chat.history.append({"role": role, "parts": [message]})

In [123]:
# Saudação inicial
print(f"Olá eu sou Lisa!👋\n Uma assistente virtual especialista em criar roteiros para retrospectivas.")
print()
print("A seguir, você pode descrever o contexto da sua equipe/time, e em seguida me fazer perguntas sobre detalhes da execução de cada uma das atividades, ok?")
print("-"*25)

# Prompting (ROTEIRO)
user_prompt = input("Descreva o contexto recente do time/equipe que deseja criar um roteiro: ")
roteiro, resultado_busca = gerar_roteiro(user_prompt)
print_roteiro(roteiro)

# Atualiza o historico do modelo generativo para ele ter contexto para iterações futuras
update_chat_history(chat, "user", f"Solicito a criação de um roteiro de retrospectiva para {user_prompt}")
update_chat_history(chat, "model", f"{roteiro}")

# Inicio do modo generativo
print("-"*25)
print("Gostou do seu roteiro? Se quiser mais detalhes sobre as atividades é só me perguntar, \nou se desejar terminar a conversa, envie a mensagem 'fim'.")
print()

# Loop do modo generativo
while True:
    # Prompting (GENERATIVO)
    user_prompt = input(">>> ")

    # Verifica se o usuário deseja terminar a conversa
    if user_prompt.strip().lower() == "fim":
        print("Até mais! 👋")
        break

    # Envia a pergunta ao modelo generativo
    resposta = chat.send_message(user_prompt)

    # Imprime a resposta do modelo
    print(resposta.text)


Olá eu sou Lisa!👋
 Uma assistente virtual especialista em criar roteiros para retrospectivas.

A seguir, você pode descrever o contexto da sua equipe/time, e em seguida me fazer perguntas sobre detalhes da execução de cada uma das atividades, ok?
-------------------------
Descreva o contexto recente do time/equipe que deseja criar um roteiro: Tenho um time recém criado que passou por muitas mudanças e seus membros precisam se reconectar uns com os outros
# Retrospectiva (roteiro)

## Abertura
**Duração:** 10 min

**Formação:** Duplas

**Atividade:** Desenho colaborativo

Em duplas, as pessoas se revezam completando um desenho.

## Reflexão e Colhendo Dados
**Duração:** 10 min

**Formação:** Individual

**Atividade:** Mad Sad Glad

As pessoas escrevem o que as deixou felizes, tristes e irritadas durante a sprint.

## Decidindo o que fazer
**Duração:** 5 min

**Formação:** Individual

**Atividade:** Dot Voting

Votar nos tópicos mais importantes para serem discutidos.

## Fechamento
**Du

In [119]:
contexto1 = "Tenho um time recém criado que passou por muitas mudanças e seus membros precisam se reconectar uns com os outros"
contexto2 = "Tenho um time experiente que apesar do bom clima interperssoal não expõem seus problemas de maneira clara e construtiva"
contexto3 = "Tenho um time experiente que precisa fazer melhorias no seu processo de trabalho para realizar entregas importantes num futuro breve"
prompt1 = "A equipe está com dificuldade em se comunicar e tomar decisões conjuntas."
prompt2 = "Um time que precisa formular novas idéias para construção do seu roadmap"
prompt3 = "Um time está com dificuldade para encontrar o real motivo dos principais problemas a sua produtividade"
prompt4 = "O time precisa melhorar as relações entre seus integrantes"